In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1500,
  chunk_overlap=200,
  add_start_index=True
)

loader = Docx2txtLoader("./tax.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)

document_list


In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from dotenv import load_dotenv

tax_db_path = "./tax_db"
tax_collection_name = "tax"

load_dotenv()
embedding_func = OpenAIEmbeddings(model="text-embedding-3-large")
tax_chroma_db = Chroma(persist_directory=tax_db_path, embedding_function=embedding_func, collection_name=tax_collection_name)


In [7]:
retrieved_docs = tax_chroma_db.similarity_search(query="연봉이 5천만원인 직장인의 소득세는 얼마인가요?", k=3)
retrieved_docs

[Document(id='71efc609-a5d7-4693-899b-a2eb81739b10', metadata={'start_index': 6612, 'source': './tax.docx'}, page_content='[전문개정 2009. 12. 31.]\n\n\n\n제10조(납세지의 변경신고) 거주자나 비거주자는 제6조부터 제9조까지의 규정에 따른 납세지가 변경된 경우 변경된 날부터 15일 이내에 대통령령으로 정하는 바에 따라 그 변경 후의 납세지 관할 세무서장에게 신고하여야 한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제11조(과세 관할) 소득세는 제6조부터 제10조까지의 규정에 따른 납세지를 관할하는 세무서장 또는 지방국세청장이 과세한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2장 거주자의 종합소득 및 퇴직소득에 대한 납세의무 <개정 2009. 12. 31.>\n\n\n\n제1절 비과세 <개정 2009. 12. 31.>\n\n\n\n제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31., 2024. 12. 31.>\n\n1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시

In [ ]:
# tax_document = Docx2txtLoader("./tax.docx").load()
# splitted_documents = text_splitter.split_documents(tax_document)
# splitted_documents

# splitted_documents = Docx2txtLoader("./tax.docx").load_and_split(text_splitter=text_splitter)

In [ ]:
query = "연봉이 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_context = tax_chroma_db.similarity_search(query=query, k=4)

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

prompt = f"""
[Identity]
- 당신은 최고의 한국 소득세 전문가입니다.
- [Context]를 참고해서 질문에 답변해주세요.

[Context]
{retrieved_context}

Question: {query}
"""

answer = llm.invoke(prompt)

In [23]:
answer.content

'소득세는 다양한 요소에 따라 달라지며, 연봉뿐만 아니라 공제, 세율, 세액공제 등을 고려하여 정확한 계산이 필요합니다. 한국의 소득세율은 누진세 구조를 따르며, 과세표준에 따라 다른 세율이 적용됩니다. \n\n연봉이 5천만원인 직장인의 소득세를 대략적으로 계산하기 위해서는 아래와 같은 기본적인 단계를 참고할 수 있습니다:\n\n1. **과세표준 계산**:\n   - 총 급여에서 근로소득공제, 연금소득공제 등을 차감하여 과세표준을 계산합니다.\n\n2. **세율 적용**:\n   - 계산된 과세표준에 해당하는 세율을 적용합니다. 한국의 소득세율은 단계별로 다른 세율이 적용되며, 과세표준이 높을수록 세율이 높아집니다. \n\n3. **세액공제**:\n   - 산출된 세액에서 연말정산에 따른 세액공제를 적용하여 최종 소득세를 계산합니다.\n\n일반적으로 연봉 5천만원인 경우 다음 항목들을 고려할 수 있습니다:\n- 근로소득공제: 일정 비율로 공제, 최대 2천만원까지\n- 세율구간: 과세표준에 따라 6%, 15%, 24% 등의 세율이 적용됨\n- 세액공제: 공제 가능한 항목에 따라 달라집니다\n\n보다 정확한 계산을 위해 소득세 계산기를 사용하거나, 세제 전문가와 상담하는 것이 좋습니다. 구체적인 항목별 공제와 세율, 공제 항목 등에 따라 최종 세액이 달라질 수 있기 때문입니다.'

In [ ]:


query = "연봉이 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_context = tax_chroma_db.similarity_search(query=query, k=4)

retrieved_context

In [4]:
from langchain_chroma import Chroma

tax_db_path = "./tax_db"


tempDB = Chroma(persist_directory=tax_db_path)
collections = [collection.name for collection in tempDB._client.list_collections()]

collections

['tax', 'langchain']

In [4]:
import os
from typing import List
from langchain.schema import Document
from langchain_community.document_loaders import Docx2txtLoader
from langchain import hub
from langchain.chains import RetrievalQA
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from dotenv import load_dotenv

def setup_chroma_db(
    persist_directory: str = "./tax_db",
    collection_name: str = "tax",
    splitted_documents: List[Document] = "./tax.docx",
) -> Chroma:
  """
  collection_name이 존재하지 않으면 새로 생성하고, 존재하면 기존 Chroma DB를 반환
  
  Args:
        persist_directory: 벡터 DB 저장 경로
        collection_name: 컬렉션 이름
        splitted_documents: 텍스트 청크 리스트
    
  Returns:
      Chroma: 설정된 Chroma 벡터 DB 인스턴스
  """

    # 기존 벡터 DB가 있는지 확인 (chroma.sqlite3 파일 존재 여부)
  chroma_db_path = os.path.join(persist_directory, "chroma.sqlite3")
  is_db_exists = os.path.exists(chroma_db_path)
  embedding = OpenAIEmbeddings(model="text-embedding-3-large")
  
  if (is_db_exists and 
    collection_name in [collection.name for collection in Chroma(persist_directory=persist_directory)._client.list_collections()]
  ):
    return Chroma(
      persist_directory=persist_directory,
      collection_name=collection_name,
      embedding_function=embedding,
    )

  return Chroma.from_documents(
    documents=splitted_documents,
    embedding=embedding,
    persist_directory=persist_directory,
    collection_name=collection_name
  )
  
TAX_DB_PATH = "./tax_db"
TAX_COLLECTION_NAME = "tax"

load_dotenv()

splitted_documents = Docx2txtLoader("./tax.docx").load_and_split(
  text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    add_start_index=True
  )
)

len(splitted_documents)

tax_chroma_db = setup_chroma_db(
  splitted_documents=splitted_documents, 
  persist_directory=TAX_DB_PATH, 
  collection_name=TAX_COLLECTION_NAME
)

qa_chain = RetrievalQA.from_chain_type(
  llm=ChatOpenAI(model="gpt-4o"),
  retriever=tax_chroma_db.as_retriever(search_kwargs={"k": 2}),
  chain_type_kwargs={"prompt": hub.pull("rlm/rag-prompt")}
)

query = "연봉이 5천만원인 직장인의 소득세는 얼마인가요?"
result = qa_chain({"query": query})

result

/Users/pgj/.pyenv/versions/3.13.3/envs/inflearn-rag-llm-application/lib/python3.13/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


{'query': '연봉이 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '이 문맥에서는 특정 연봉에 따른 소득세율이나 계산 방법이 제공되지 않았습니다. 연봉이 5천만 원인 직장인의 소득세를 계산하려면 세율표와 관련 공제를 확인해야 합니다. 더욱 구체적인 정보가 필요합니다.'}

# cosine smiliarity

In [1]:
%pip instsall python-dotenv

ERROR: unknown command "instsall" - maybe you meant "install"
Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv
load_dotenv()


True

In [3]:
from openai import OpenAI
client = OpenAI()

In [4]:
import numpy as np

def cosine_similarity(vec1, vec2):
  """
  Calculate the cosine similarity between two vectors.

  Parameters:
  vec1 (numpy array): First vector
  vec2 (numpy array): second vector
  
  Returns:
  float: Cosine similarity between vec1 and vec2
  """
  dot_product = np.dot(vec1, vec2)
  norm_vec1 = np.linalg.norm(vec1)
  norm_vec2 = np.linalg.norm(vec2)

  if norm_vec1 == 0 or norm_vec2 == 0:
    return 0.0
  
  return dot_product / (norm_vec1 * norm_vec2)

In [19]:
king_embedded_response = client.embeddings.create(input="king", model="text-embedding-3-large")
queen_embedded_response = client.embeddings.create(input="queen", model="text-embedding-3-large")

king_vector = np.array(king_embedded_response.data[0].embedding)
queen_vector = np.array(queen_embedded_response.data[0].embedding)

In [20]:
king_queen_similarity = cosine_similarity(king_vector, queen_vector)
print(king_queen_similarity)

0.5552268369726672
